In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

In [3]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

In [4]:
train_images = train_images[:1000]
test_images = test_images[:1000]

In [5]:
train_images = train_images.reshape(-1, 28 * 28) / 255.

In [6]:
test_images = test_images.reshape(-1, 28 * 28) / 255.

In [7]:
def create_model():
    model = Sequential([
        Dense(512, activation='relu', input_shape=(test_images.shape[1:])),
        Dropout(0.2),
        Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[keras.metrics.SparseCategoricalAccuracy()])
    return model

In [8]:
model = create_model()

Metal device set to: Apple M1


2022-07-03 14:50:57.509529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-03 14:50:57.510053: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### keras.callbacks.ModelCheckpoint saves weights only during traning

In [9]:
!mkdir training_1
checkpoint_path = 'training_1/training_1_cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

mkdir: training_1: File exists


#### Create a callback that saves the model's weights

In [11]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                              save_weights_only=True, verbose=0)

#### Train the model with the new callback

In [12]:
model.fit(train_images, 
          train_labels, 
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10


2022-07-03 14:52:43.296645: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 9/32 [=======>......................] - ETA: 0s - loss: 1.8889 - sparse_categorical_accuracy: 0.3715

2022-07-03 14:52:43.520488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 13ms/step - loss: 1.1871 - sparse_categorical_accuracy: 0.6330 - val_loss: 0.7404 - val_sparse_categorical_accuracy: 0.7780
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.5040 - sparse_categorical_accuracy: 0.9062

2022-07-03 14:52:43.893575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 9ms/step - loss: 0.4160 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5837 - val_sparse_categorical_accuracy: 0.8130
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 0.2975 - sparse_categorical_accuracy: 0.9170 - val_loss: 0.4831 - val_sparse_categorical_accuracy: 0.8530
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 0.2072 - sparse_categorical_accuracy: 0.9490 - val_loss: 0.4388 - val_sparse_categorical_accuracy: 0.8680
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 0.1448 - sparse_categorical_accuracy: 0.9700 - val_loss: 0.4434 - val_sparse_categorical_accuracy: 0.8570
Epoch 6/10
32/32 [==============================] - 0s 9ms/step - loss: 0.1215 - sparse_categorical_accuracy: 0.9770 - val_loss: 0.4227 - val_sparse_categorical_accuracy: 0.8640
Epoch 7/10
32/32 [==============================] - 0s 9ms/step - loss: 0.0838 - sparse_categorical_accuracy: 0.9890 - va

### from_logits의 경우 class 분류의 경우 true가 regression 같은 경우는 false가 나은 듯

In [13]:
os.listdir(checkpoint_dir)

['.DS_Store',
 'checkpoint',
 'training_1.index',
 'tutorial Save and load models.ipynb',
 'training_1.data-00000-of-00001']

### make new model and evaluate, after load callback and evaluate

In [14]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Unrestored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 2.4014 - sparse_categorical_accuracy: 0.1040 - 156ms/epoch - 5ms/step
Unrestored model, accuracy: 10.40%


2022-07-03 14:54:07.499177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [17]:
model.load_weights(checkpoint_path)

In [18]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4207 - sparse_categorical_accuracy: 0.8660 - 112ms/epoch - 3ms/step
Restored model, accuracy: 86.60%


In [33]:
!mkdir training_2
checkpoint_path = 'training_2/training_2-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

mkdir: training_2: File exists


In [34]:
batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                verbose=1,
                                                save_weights_only=True,
                                                save_freq=5*batch_size)

In [35]:
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels, epochs=50, batch_size=batch_size, 
         callbacks=[cp_callback], validation_data=(test_images, test_labels),
         verbose=0)

2022-07-03 15:03:11.826650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-03 15:03:12.082630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 5: saving model to training_2/training_2-0005.ckpt

Epoch 10: saving model to training_2/training_2-0010.ckpt

Epoch 15: saving model to training_2/training_2-0015.ckpt

Epoch 20: saving model to training_2/training_2-0020.ckpt

Epoch 25: saving model to training_2/training_2-0025.ckpt

Epoch 30: saving model to training_2/training_2-0030.ckpt

Epoch 35: saving model to training_2/training_2-0035.ckpt

Epoch 40: saving model to training_2/training_2-0040.ckpt

Epoch 45: saving model to training_2/training_2-0045.ckpt

Epoch 50: saving model to training_2/training_2-0050.ckpt


In [36]:
os.listdir(checkpoint_dir)

['training_2-0005.ckpt.index',
 'training_2-0050.ckpt.index',
 'training_2-0025.ckpt.data-00000-of-00001',
 '.DS_Store',
 'checkpoint',
 'training_2-0000.ckpt.index',
 'training_2-0000.ckpt.data-00000-of-00001',
 'training_2-0050.ckpt.data-00000-of-00001',
 'training_2-0030.ckpt.index',
 'training_2-0005.ckpt.data-00000-of-00001',
 'training_2-0035.ckpt.index',
 'training_2-0020.ckpt.data-00000-of-00001',
 'training_2-0030.ckpt.data-00000-of-00001',
 'training_2-0045.ckpt.index',
 'training_2-0010.ckpt.index',
 'training_2-0015.ckpt.data-00000-of-00001',
 'training_2-0040.ckpt.index',
 'training_2-0040.ckpt.data-00000-of-00001',
 'training_2-0015.ckpt.index',
 'training_2-0010.ckpt.data-00000-of-00001',
 'training_2-0025.ckpt.index',
 'training_2-0045.ckpt.data-00000-of-00001',
 'training_2-0035.ckpt.data-00000-of-00001',
 'training_2-0020.ckpt.index']

In [37]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [38]:
latest

'training_2/training_2-0050.ckpt'

In [39]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('{0:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4859 - sparse_categorical_accuracy: 0.8750 - 153ms/epoch - 5ms/step
87.50%


2022-07-03 15:03:41.865662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [40]:
weights_list = os.listdir(checkpoint_dir)

In [41]:
weights_list.sort()

In [42]:
weights_list = [file for file in weights_list if 'training_2' in file]

In [44]:
for i in range(int(len(weights_list) / 2)):
    files = '{}{:0>4}.ckpt'.format('training_2-', i * 5)
    files = os.path.join('training_2', files)
    model.load_weights(files)
    loss, acc = model.evaluate(test_images, test_labels, verbose=0)
    print('accuracy {:2}: {:5.2f}'.format(i * 5, 100 * acc))

accuracy  0: 10.40
accuracy  5: 87.00
accuracy 10: 86.30
accuracy 15: 87.60
accuracy 20: 87.70
accuracy 25: 87.80
accuracy 30: 87.80
accuracy 35: 87.70
accuracy 40: 87.50
accuracy 45: 87.60
accuracy 50: 87.50


In [45]:
model.save_weights('./checkpoints/my_checkpoint')

In [46]:
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

In [47]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [48]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('restored model: {:5.2f}'.format(100 * acc))

32/32 - 0s - loss: 0.4859 - sparse_categorical_accuracy: 0.8750 - 310ms/epoch - 10ms/step
restored model: 87.50


2022-07-03 15:05:23.305227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [49]:
model = create_model()
model.fit(train_images, train_labels, epochs=5, verbose=0)

2022-07-03 15:07:15.931055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [50]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


2022-07-03 15:07:42.384163: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [51]:
# my_model directory
!ls saved_model

my_model


In [52]:
# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

assets            keras_metadata.pb saved_model.pb    variables


In [55]:
new_model = keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [57]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('restored model, accuracy: {:5.2f}%'.format(acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4268 - sparse_categorical_accuracy: 0.8680 - 118ms/epoch - 4ms/step
restored model, accuracy:  0.87%
(1000, 10)


2022-07-03 15:17:58.147710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### HDF5 format

In [66]:
model = create_model()
model.summary()
model.fit(train_images, train_labels, epochs=5, verbose=2)

model.save('my_model.h5')

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 512)               401920    
                                                                 
 dropout_16 (Dropout)        (None, 512)               0         
                                                                 
 dense_33 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2022-07-03 15:24:03.995847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32/32 - 0s - loss: 1.1787 - sparse_categorical_accuracy: 0.6610 - 327ms/epoch - 10ms/step
Epoch 2/5
32/32 - 0s - loss: 0.4374 - sparse_categorical_accuracy: 0.8710 - 133ms/epoch - 4ms/step
Epoch 3/5
32/32 - 0s - loss: 0.2883 - sparse_categorical_accuracy: 0.9230 - 133ms/epoch - 4ms/step
Epoch 4/5
32/32 - 0s - loss: 0.2041 - sparse_categorical_accuracy: 0.9470 - 135ms/epoch - 4ms/step
Epoch 5/5
32/32 - 0s - loss: 0.1570 - sparse_categorical_accuracy: 0.9640 - 141ms/epoch - 4ms/step


In [67]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)

32/32 - 0s - loss: 0.4578 - sparse_categorical_accuracy: 0.8500 - 217ms/epoch - 7ms/step


2022-07-03 15:24:40.444001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [65]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 512)               401920    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [68]:
lass, acc = model.evaluate(test_images, test_labels, verbose=2)

32/32 - 0s - loss: 0.4578 - sparse_categorical_accuracy: 0.8500 - 113ms/epoch - 4ms/step
